# Day 2: 1202 Program Alarm

[Aoc Site](https://adventofcode.com/2019/day/2)

## Part I

On the way to your gravity assist around the Moon, your ship computer beeps angrily about a "1202 program alarm". On the radio, an Elf is already explaining how to handle the situation: "Don't worry, that's perfectly norma--" The ship computer bursts into flames.

You notify the Elves that the computer's magic smoke seems to have escaped. "That computer ran Intcode programs like the gravity assist program it was working on; surely there are enough spare parts up there to build a new Intcode computer!"

An Intcode program is a list of integers separated by commas (like 1,0,0,3,99). To run one, start by looking at the first integer (called position 0). Here, you will find an opcode - either 1, 2, or 99. The opcode indicates what to do; for example, 99 means that the program is finished and should immediately halt. Encountering an unknown opcode means something went wrong.

**Opcode 1** adds together numbers read from two positions and stores the result in a third position. The three integers immediately after the opcode tell you these three positions - the first two indicate the positions from which you should read the input values, and the third indicates the position at which the output should be stored.

For example, if your Intcode computer encounters 1,10,20,30, it should read the values at positions 10 and 20, add those values, and then overwrite the value at position 30 with their sum.

**Opcode 2** works exactly like opcode 1, except it multiplies the two inputs instead of adding them. Again, the three integers after the opcode indicate where the inputs and outputs are, not their values.

**Once you're done processing an opcode, move to the next one by stepping forward 4 positions.**

For example, suppose you have the following program:

**1,9,10,3,2,3,11,0,99,30,40,50**

For the purposes of illustration, here is the same program split into multiple lines:

- **1,9,10,3,**
- **2,3,11,0,**
- **99,**
- **30,40,50**

The first four integers, **1,9,10,3,** are at positions 0, 1, 2, and 3. Together, they represent the first opcode (1, addition), the positions of the two inputs (9 and 10), and the position of the output (3). To handle this opcode, you first need to get the values at the input positions: position 9 contains 30, and position 10 contains 40. Add these numbers together to get 70. Then, store this value at the output position; here, the output position (3) is at position 3, so it overwrites itself. Afterward, the program looks like this:

- **1,9,10,70,**
- **2,3,11,0,**
- **99,**
- **30,40,50**

Step forward 4 positions to reach the next opcode, 2. This opcode works just like the previous, but it multiplies instead of adding. The inputs are at positions 3 and 11; these positions contain 70 and 50 respectively. Multiplying these produces 3500; this is stored at position 0:

- **3500,9,10,70,**
- **2,3,11,0,**
- **99,**
- **30,40,50**

Stepping forward 4 more positions arrives at opcode 99, halting the program.

Here are the initial and final states of a few more small programs:

- **1,0,0,0,99** becomes **2,0,0,0,99** (1 + 1 = 2).
- **2,3,0,3,99** becomes **2,3,0,6,99** (3 * 2 = 6).
- **2,4,4,5,99,0** becomes **2,4,4,5,99,9801** (99 * 99 = 9801).
- **1,1,1,4,99,5,6,0,99** becomes **30,1,1,4,2,5,6,0,99**.

#### Once you have a working computer, the first step is to restore the gravity assist program (your puzzle input) to the "1202 program alarm" state it had just before the last computer caught fire. To do this, before running the program, replace position 1 with the value 12 and replace position 2 with the value 2. What value is left at position 0 after the program halts?

To begin, [get your puzzle input](https://adventofcode.com/2019/day/2/input).

In [1]:
# Prep Notebook
import pandas as pd
import numpy as np

import disfunctions as func

In [2]:
# Create test DataFrame
input_col = 'input'
target1_col = 'target_p1'
output1_col = 'output_p1'

input_s = pd.Series([[1,0,0,0,99], [2,3,0,3,99], [2,4,4,5,99,0], [1,1,1,4,99,5,6,0,99]], name=input_col)
target1_s = pd.Series([[2,0,0,0,99], [2,3,0,6,99], [2,4,4,5,99,9801], [30,1,1,4,2,5,6,0,99]], name=target1_col)

test_df = pd.DataFrame([input_s, target1_s]).T

test_df

,input,target_p1
0,"[1, 0, 0, 0, 99]","[2, 0, 0, 0, 99]"
1,"[2, 3, 0, 3, 99]","[2, 3, 0, 6, 99]"
2,"[2, 4, 4, 5, 99, 0]","[2, 4, 4, 5, 99, 9801]"
3,"[1, 1, 1, 4, 99, 5, 6, 0, 99]","[30, 1, 1, 4, 2, 5, 6, 0, 99]"


In [3]:
# Functionize opcode1

def run_opcode_step(code_list, start_pos, show_it=False):
    run_code = code_list[start_pos]
    max_pos = len(code_list)
    var1_pos = code_list[min(start_pos + 1, max_pos-1)]
    var1 = code_list[var1_pos]
    var2_pos = code_list[min(start_pos + 2, max_pos-1)]
    var2 = code_list[var2_pos]
    answer_pos = code_list[min(start_pos + 3, max_pos-1)]
    if show_it:
        print(f'run_code: {run_code}, var1: {var1}, var2: {var2}:, answer_pos: {answer_pos}')
    prev_value = code_list[answer_pos]
    if run_code == 99:
        if show_it:
            print(f'End of code at step {start_pos}')
        return 0
    if run_code == 1:
        result = var1 + var2
    elif run_code == 2:
         result = var1 * var2
    else:
        print(f'ERROR FOUND at step {start_pos}')
        return -1
    code_list[answer_pos] = result
    if show_it:
        print(f'Position {answer_pos} changed to {result}')
    return 1

In [4]:
# Functionize movement

def opcode_step_through(start_codes, show_it=False):
    code_list = start_codes.copy()
#     if show_it:
#         print(code_list)
    on_pos = 0
    max_pos = len(code_list)
    go_code = 1
    steps = 0
    while go_code > 0:
        to_pos = min(on_pos + 4, max_pos-1)
        if code_list[on_pos] == 99:
            go_code = 0
#             if show_it:
#                 print('Result:', code_list)
            continue
        if show_it:
#             print(on_pos, to_pos)
            print(f'Position {on_pos}:') 
            print(code_list[on_pos:to_pos])
        go_code = run_opcode_step(code_list, on_pos, show_it=show_it)
#         if show_it:
#             print(f'Position {on_pos}:', code_list[on_pos:to_pos])
#             print('Go Code =', go_code)
        on_pos = to_pos
        steps += 1
    print(f'Resolved in {steps} steps')
    return code_list

In [5]:
def opcode_step_through_df(df, input_col='input', output_col='output', show_it=False):
    df[output_col] = df[input_col].apply(lambda x: opcode_step_through(x, show_it=show_it))
    return df

In [6]:
# Run test P1

result1_col = 'result_P1'

# test_df[output1_col] = test_df[input_col].apply(lambda x: opcode_step_through(x, show_it=False))
test_df = opcode_step_through_df(test_df, input_col=input_col, output_col=output1_col)
result_p1 = func.test_target_check(test_df, target_col=target1_col, test_col=output1_col, result_col=result1_col)
display(test_df)
print('Result:', result_p1)

Resolved in 1 steps
Resolved in 1 steps
Resolved in 1 steps
Resolved in 2 steps


,input,target_p1,output_p1,result_P1
0,"[1, 0, 0, 0, 99]","[2, 0, 0, 0, 99]","[2, 0, 0, 0, 99]",True
1,"[2, 3, 0, 3, 99]","[2, 3, 0, 6, 99]","[2, 3, 0, 6, 99]",True
2,"[2, 4, 4, 5, 99, 0]","[2, 4, 4, 5, 99, 9801]","[2, 4, 4, 5, 99, 9801]",True
3,"[1, 1, 1, 4, 99, 5, 6, 0, 99]","[30, 1, 1, 4, 2, 5, 6, 0, 99]","[30, 1, 1, 4, 2, 5, 6, 0, 99]",True


Result: True


#### Get source data

In [7]:
# Read Source
src_df = pd.read_csv('2019_02.txt', header=None).T.rename(columns={0:'code'})
src_df.head()

,code
0,1
1,0
2,0
3,3
4,1


#### Perform initial steps

In [8]:
p1_df = src_df.copy()

In [9]:
p1_df.iloc[1]=12
p1_df.iloc[2]=2
p1_df.head()

,code
0,1
1,12
2,2
3,3
4,1


In [10]:
src_df.head()

,code
0,1
1,0
2,0
3,3
4,1


In [11]:
p1_df['result_p1'] = opcode_step_through(p1_df.code, show_it=False)
p1_df.head(8)

Resolved in 28 steps


,code,result_p1
0,1,3895705
1,12,12
2,2,2
3,3,2
4,1,1
5,1,1
6,2,2
7,3,3


In [12]:
answer_p1 = p1_df.result_p1[0]

In [13]:
actual_p1 = 3895705
print(actual_p1==answer_p1)

True


### That's the right answer! You are one gold star closer to rescuing Santa. 

## Part II

"Good, the new computer seems to be working correctly! Keep it nearby during this mission - you'll probably use it again. Real Intcode computers support many more features than your new one, but we'll let you know what they are as you need them."

"However, your current priority should be to complete your gravity assist around the Moon. For this mission to succeed, we should settle on some terminology for the parts you've already built."

Intcode programs are given as a list of integers; these values are used as the initial state for the computer's memory. When you run an Intcode program, make sure to start by initializing memory to the program's values. A position in memory is called an address (for example, the first value in memory is at "address 0").

Opcodes (like 1, 2, or 99) mark the beginning of an instruction. The values used immediately after an opcode, if any, are called the instruction's parameters. For example, in the instruction 1,2,3,4, 1 is the opcode; 2, 3, and 4 are the parameters. The instruction 99 contains only an opcode and has no parameters.

The address of the current instruction is called the instruction pointer; it starts at 0. After an instruction finishes, the instruction pointer increases by the number of values in the instruction; until you add more instructions to the computer, this is always 4 (1 opcode + 3 parameters) for the add and multiply instructions. (The halt instruction would increase the instruction pointer by 1, but it halts the program instead.)

"With terminology out of the way, we're ready to proceed. To complete the gravity assist, you need to determine what pair of inputs produces the output 19690720."

The inputs should still be provided to the program by replacing the values at addresses 1 and 2, just like before. In this program, the value placed in address 1 is called the noun, and the value placed in address 2 is called the verb. Each of the two input values will be between 0 and 99, inclusive.

Once the program has halted, its output is available at address 0, also just like before. Each time you try a pair of inputs, make sure you first reset the computer's memory to the values in the program (your puzzle input) - in other words, don't reuse memory from a previous attempt.

Find the input noun and verb that cause the program to produce the output 19690720. What is 100 * noun + verb? (For example, if noun=12 and verb=2, the answer would be 1202.)

In [14]:
# Functionize Part 1

def compute_intcode(src_df, noun=12, verb=2, code_col='code', result_col='result', show_it=False):
    df = src_df.copy()
    df.iloc[1]=noun
    df.iloc[2]=verb
    df[result_col] = opcode_step_through(df[code_col], show_it=show_it)
    return df

In [15]:
target2 = 19690720

In [16]:
noun=64
verb=17
code = str(noun)+str(verb)
df_check = compute_intcode(src_df, noun=64, verb=17, code_col='code', result_col='result', show_it=True)
display(df_check.head(4))
result2 = df_check.result[0]
print(code, target2, result2, target2-result2)

Position 0:
0     1
1    64
2    17
3     3
Name: code, dtype: int64
run_code: 1, var1: 1, var2: 1:, answer_pos: 3
Position 3 changed to 2
Position 4:
4    1
5    1
6    2
7    3
Name: code, dtype: int64
run_code: 1, var1: 64, var2: 17:, answer_pos: 3
Position 3 changed to 81
Position 8:
8     1
9     3
10    4
11    3
Name: code, dtype: int64
run_code: 1, var1: 81, var2: 1:, answer_pos: 3
Position 3 changed to 82
Position 12:
12    1
13    5
14    0
15    3
Name: code, dtype: int64
run_code: 1, var1: 1, var2: 1:, answer_pos: 3
Position 3 changed to 2
Position 16:
16     2
17     1
18     9
19    19
Name: code, dtype: int64
run_code: 2, var1: 64, var2: 3:, answer_pos: 19
Position 19 changed to 192
Position 20:
20     1
21    19
22     5
23    23
Name: code, dtype: int64
run_code: 1, var1: 192, var2: 1:, answer_pos: 23
Position 23 changed to 193
Position 24:
24     2
25    23
26    13
27    27
Name: code, dtype: int64
run_code: 2, var1: 193, var2: 5:, answer_pos: 27
Position 27 changed 

,code,result
0,1,19690720
1,64,64
2,17,17
3,3,2


6417 19690720 19690720 0


In [17]:
actual_p2 = 6417

In [18]:
result2==actual_p2

False

### That's the right answer! You are one gold star closer to rescuing Santa.